# Random Forest with 18 variables
아래 코드는 논문 작성 및 마지막 정이를 위해 작성된 파일임.


Input data:


Output data:


In [7]:
import sys, os
def importOwnLib():
    if '/home/miruware/aProjects/lib' not in sys.path:
        sys.path.append('/home/miruware/aProjects/lib')
        print("lib path is successfully appended.")
    else:
        print("lib path is already exists.")

importOwnLib()

import sonyalib as sonya
import importlib
importlib.reload(sonya)
import matplotlib.pyplot as plt


import tensorflow as tf
import statistics
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV,StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from pprint import pprint
from sklearn.feature_selection import SelectFromModel
# from sklearn.feature_selection import RFECV, RFE, SelectFromModel
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier
# from sklearn.pipeline import Pipeline
# from sklearn.metrics import auc, roc_curve
# from scipy import interp
# import keras
# # from keras import callbacks, losses
# from keras.optimizers import Adam
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, BatchNormalization, Dropout
# from tensorflow.python.keras._impl.keras import Model
# from tensorflow.python.keras._impl.keras.models import Model
from keras import initializers
from keras import losses, callbacks
from keras.optimizers import adam

lib path is already exists.


### Hyper parameter tuning - random grid search

In [27]:
# fit random seed for reproductivity
seed = 7
np.random.seed(seed)

# Load metadata
file_path_train = './BRC_input_201116_train.xlsx' # train
file_path_test = './BRC_input_201116_test.xlsx' # test
meta_train = sonya.get_normalized_metadata(file_path_train)
meta_test = sonya.get_normalized_metadata(file_path_test)
properties = [
    'sex',
    'age',
    'LR',
    'cT',
    'cN',
    'cAverage',
    'cSD',
    'aAverage',
    'aSD',
    'lMax',
    'homogeneous',
    'hetero',
    'rim',
    'clustered',
    'non-mass',
    'AorCa',
    'LymAo',
    'LymCa'
]

num_properties = len(properties)

X_train = meta_train[properties]
y_train = meta_train['label']
X_test = meta_test[properties]
y_test = meta_test['label']

In [6]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (196, 18)
Training Labels Shape: (196,)
Testing Features Shape: (84, 18)
Testing Labels Shape: (84,)


### Examine the Default Random Forest to Determine Parameters

In [4]:
rf_clf = RandomForestClassifier(random_state=42)

print('Parameters currently in use: \n')
pprint(rf_clf.get_params())

Parameters currently in use: 

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 10,
 'n_jobs': 1,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


### Grid Search with Cross Validation

In [7]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 800, num = 8)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(start = 5, stop = 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
rf_param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

pprint(rf_param_grid)

{'bootstrap': [True, False],
 'max_depth': [5, 10, 15, 20, 25, 30],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10, 15, 100],
 'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800]}


In [9]:
# kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
# rf = RandomForestClassifier(n_estimators=10)
# accuracy = np.mean(cross_val_score(rf, X_train, y_train, scoring="accuracy", cv=kfold))
# print(accuracy)

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf_clf = RandomForestClassifier(random_state = 42)
# Grid search of parameters, using 5 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = GridSearchCV(estimator=rf_clf, 
                         param_grid=rf_param_grid,
                         scoring='accuracy',
                         n_jobs=-1,
                         cv = 5, 
                         verbose=2, 
                         return_train_score=True)

# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 2880 candidates, totalling 14400 fits
[CV] bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100 
[CV] bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100 
[CV] bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100 
[CV] bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100 
[CV] bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100 
[CV] bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200 
[CV] bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200 
[CV] bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200 
[CV] bootstrap=True, max_depth=5, max_

[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.8s


[CV]  bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=500, total=   0.7s
[CV] bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100 
[CV]  bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=500, total=   0.7s
[CV] bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100 
[CV]  bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=500, total=   0.7s
[CV] bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100 
[CV]  bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=500, total=   0.7s
[CV] bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=200 
[CV]  bootstrap=True, max_depth=5, max_featu

[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:    9.5s


[CV]  bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=100, n_estimators=400, total=   0.5s
[CV] bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=100, n_estimators=600 
[CV]  bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=100, n_estimators=400, total=   0.5s
[CV] bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=100, n_estimators=600 
[CV]  bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=100, n_estimators=400, total=   0.6s
[CV] bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=100, n_estimators=700 
[CV]  bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=100, n_estimators=400, total=   0.5s
[CV] bootstrap=True, max_depth=5, max_features=auto, min_samples_leaf=1, min_samples_split=100, n_estimators=700 
[CV]  bootstrap=True, max_de

...

[CV]  bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=15, n_estimators=200, total=   0.3s
[CV] bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=15, n_estimators=200 
[CV]  bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=15, n_estimators=200, total=   0.3s
[CV] bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=15, n_estimators=300 
[CV]  bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=15, n_estimators=200, total=   0.3s
[CV]  bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=15, n_estimators=200, total=   0.3s
[CV] bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=15, n_estimators=300 
[CV] bootstrap=False, max_depth=20, max_features=sqrt, min_samples_leaf=2, min_samples_split=15, n_estimators=300 
[CV]  bootstrap=Fals

[Parallel(n_jobs=-1)]: Done 13133 tasks      | elapsed: 12.3min


[CV]  bootstrap=False, max_depth=25, max_features=sqrt, min_samples_leaf=4, min_samples_split=15, n_estimators=400, total=   0.6s
[CV] bootstrap=False, max_depth=25, max_features=sqrt, min_samples_leaf=4, min_samples_split=15, n_estimators=700 
[CV]  bootstrap=False, max_depth=25, max_features=sqrt, min_samples_leaf=4, min_samples_split=15, n_estimators=400, total=   0.6s
[CV] bootstrap=False, max_depth=25, max_features=sqrt, min_samples_leaf=4, min_samples_split=15, n_estimators=700 
[CV]  bootstrap=False, max_depth=25, max_features=sqrt, min_samples_leaf=4, min_samples_split=15, n_estimators=500, total=   0.7s
[CV] bootstrap=False, max_depth=25, max_features=sqrt, min_samples_leaf=4, min_samples_split=15, n_estimators=700 
[CV]  bootstrap=False, max_depth=25, max_features=sqrt, min_samples_leaf=4, min_samples_split=15, n_estimators=500, total=   0.7s
[CV] bootstrap=False, max_depth=25, max_features=sqrt, min_samples_leaf=4, min_samples_split=15, n_estimators=700 
[CV]  bootstrap=Fals

[Parallel(n_jobs=-1)]: Done 14400 out of 14400 | elapsed: 13.5min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': [100, 200, 300, 400, 500, 600, 700, 800], 'max_features': ['auto', 'sqrt'], 'max_depth': [5, 10, 15, 20, 25, 30], 'min_samples_split': [2, 5, 10, 15, 100], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=2)

In [12]:
rf_random.best_params_

{'bootstrap': True,
 'max_depth': 15,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 100}

In [19]:
rf_clf_best = RandomForestClassifier(n_estimators = 100,
                                    max_depth = 15,
                                    max_features = 'auto',
                                    min_samples_leaf = 1,
                                    min_samples_split = 10)
scoring = "roc_auc"

score = cross_val_score(rf_clf_best, X_train, y_train, cv = 10, n_jobs=-1, scoring=scoring)

score

array([0.94047619, 0.89285714, 0.82142857, 0.89285714, 0.60714286,
       0.86904762, 0.83333333, 0.84615385, 0.79487179, 0.67692308])

In [10]:
score = [ 0.94047619, 0.89285714, 0.82142857, 0.89285714, 0.60714286,
       0.86904762, 0.83333333, 0.84615385, 0.79487179, 0.67692308 ]

In [13]:
from math import sqrt
x_bar = np.mean(score)
s = np.std(score)
n = len(score)

z = 1.96
lower = x_bar - (z * (s / sqrt(n)))
upper = x_bar + (z * (s / sqrt(n)))

In [15]:
print(s)
print(x_bar)
print(lower, upper)
# print(statistics.stdev(score))

0.09733904688687868
0.8175091570000002
0.7571777906872382 0.8778405233127621


## ======= Random Forest as Feature Selection =======

In [19]:
rf_clf_best = RandomForestClassifier(n_estimators = 100,
                                    max_depth = 15,
                                    max_features = 'auto',
                                    min_samples_leaf = 1,
                                    min_samples_split = 10)

In [21]:
sel = SelectFromModel(rf_clf_best)
sel.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier(max_depth=15,
                                                 min_samples_split=10))

In [22]:
sel.get_support()

array([False, False, False,  True,  True,  True,  True,  True, False,
        True, False, False, False, False, False,  True,  True,  True])

In [23]:
selected_feat= X_train.columns[(sel.get_support())]
len(selected_feat)

9

In [24]:
selected_feat

Index(['cT', 'cN', 'cAverage', 'cSD', 'aAverage', 'lMax', 'AorCa', 'LymAo',
       'LymCa'],
      dtype='object')

In [26]:
num_selected = len(selected_feat)

X_selected_train = meta_train[selected_feat]
y_selected_train = meta_train['label']
X_selected_test = meta_test[selected_feat]
y_selected_test = meta_test['label']

In [ ]:
cv = StratifiedKFold(n_splits = 10, random_state = 123, shuffle = True)
fprs, tprs, scores = [], [], []
results = pd.DataFrame(columns=['training_score', 'test_score'])

In [30]:
i = 0
for train_index, validation_index in kfold.split(X_selected_train, y_selected_train):
    kX_selected_train, kX_selected_test = X_selected_train.iloc[train_index], X_selected_train.iloc[validation_index]
    ky_selected_train, ky_selected_test = y_selected_train.iloc[train_index], y_selected_train.iloc[validation_index]
    
    print("==================== FOLD: {} ====================".format(i))
    
    cbks = [callbacks.LearningRateScheduler(lambda epoch: 0.001 * 0.5 ** (epoch // 2)),
            callbacks.TensorBoard(write_graph=False)]
    
    rf_clf_best.fit(kX_selected_train, ky_selected_train, validation_data=(kX_selected_test, ky_selected_test),
                   callbacks=cbks,
                   verbose=0)
    y_val_cat_prob = rf_clf_best.predict_proba(kX_selected_test)
    k_accuracy = '%.4f' % (my_model.evaluate(kX_selected_test, ky_selected_test)[1])
    accuracy.append(k_accuracy)
    
    fpr, tpr, t = roc_curve(y_train.iloc[validation_index], y_val_cat_prob)
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
    
    i = i + 1
    
    test_loss, test_acc = rf_clf_best.evaluate(kX_selected_test, ky_selected_test)
    print('\nK-fold cross validation Accuracy: {}'.format(test_acc))    
    
    
y_pred_proba = (my_model.predict(X_train) >= 0.693).astype(int)
cm = confusion_matrix(y_train, y_pred_proba)
print("Radiologist + AI")
print(cm)
confusion_metrics(cm)

mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
roc_result = mean_auc

current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
model_name = target_dir + '/' + 'AUC' + str(int(float(mean_auc) * 100)) + current_time
model_json = my_model.to_json()

# with open('{}.json'.format(model_name), 'w') as json_file:
#     json_file.write(model_json)  # save model per fold

# my_model.save_weights('{}.h5'.format(model_name))  # save weight per fold

# #     plt.plot(fpr, tpr, color='red', label='Radiologist + AI(AUC = %0.2f)' % (mean_auc), lw=2, alpha=1)

plt.plot(mean_fpr, mean_tpr, color='red', label=r'radiologist + AI(AUC = %0.2f)' % (mean_auc), lw=2, alpha=1)    

==================== FOLD: 0 ====================


TypeError: fit() got an unexpected keyword argument 'validation_data'

In [ ]:
seed = 7
np.random.seed(seed)

In [35]:
aucs = cross_val_score(rf_clf_best, X_selected_train, y_selected_train, cv=kfold)

In [36]:
aucs

array([0.8       , 0.8       , 0.8       , 0.6       , 0.75      ,
       0.85      , 0.68421053, 0.68421053, 0.84210526, 0.78947368])

In [40]:
x_bar = np.mean(aucs)
s = np.std(aucs)
n = len(aucs)

z = 1.96
lower = x_bar - (z * (s / sqrt(n)))
upper = x_bar + (z * (s / sqrt(n)))
# interval = z * sqrt( (test_acc * (1 - test_acc)) / len(meta_test))
print(lower, x_bar, upper)
print(s)

0.712919975476416 0.76 0.8070800245235841
0.07595923968934144
